### Integração Numérica - Regra de 1/3 de Simpson

Este notebook apresenta a implementação da Regra de 1/3 de Simpson para o cálculo de integrais definidas, com cálculo do erro de aproximação simples e generalizado, utilizando orientação a objetos com uma classe base comum.


In [1]:
%load_ext autoreload
%autoreload 2
from integral import Integral
import math
import sympy as sp

### Classe `IntegralUmTercoSimpson`

In [2]:
class IntegralUmTercoSimpson(Integral):
    def __init__(self, x=None, y=None, limite_inf=None, limite_sup=None,
                 func_integral=None, h=None, erro_desejado=None):
        super().__init__()

        if erro_desejado is not None:
            if limite_inf is None or limite_sup is None or func_integral is None:
                raise ValueError("Para usar erro_desejado, forneça limite_inf, limite_sup e func_integral.")

            self.limite_inf = limite_inf
            self.limite_sup = limite_sup
            self.func_expr = func_integral  # guardar expressão simbólica

            x_sym = sp.Symbol('x')
            func_lambd = sp.lambdify(x_sym, func_integral, modules=["math"])

            # Obtém K = máximo valor absoluto da 4ª derivada em [a,b]
            K = self._derivada_quarta_simbolica(func_integral, limite_inf, limite_sup)

            if K is None:
                raise ValueError("Não foi possível calcular a quarta derivada simbolicamente.")

            n_quarta_min = ((K * (limite_sup - limite_inf) ** 5) / (180 * erro_desejado)) ** 0.25
            n = math.ceil(n_quarta_min)

            # n deve ser par para Simpson 1/3
            if n % 2 != 0:
                n += 1

            n = min(max(n, 2), 10000)  # limite mínimo e máximo razoável
            print(f"valor de n (subintervalos) encontrado: {n}")
            h = (limite_sup - limite_inf) / n
            print(f"valor de h calculado: {h}")

            self._setup(x, y, limite_inf, limite_sup, func_lambd, h)

        else:
            self._setup(x, y, limite_inf, limite_sup, func_integral, h)

    def _derivada_quarta_simbolica(self, func_expr, a, b):
        x = sp.Symbol('x')
        try:
            f_diff4 = sp.diff(func_expr, x, 4)
            f_diff4_func = sp.lambdify(x, f_diff4, modules=["math"])
            pontos = [a, (a + b) / 2, b]
            return max(abs(f_diff4_func(xi)) for xi in pontos)
        except Exception as e:
            print(f"[sympy fallback] Derivada quarta simbólica falhou: {e}")
            return None

    def integral(self):
        n = len(self.x) - 1
        soma = self.y[0] + self.y[-1]

        for i in range(1, n):
            if i % 2 == 0:
                soma += 2 * self.y[i]
            else:
                soma += 4 * self.y[i]

        return (self.h / 3) * soma

    def erro_simples(self):
        d4_inf = self.derivada_quarta_ordem(self.limite_inf)
        d4_sup = self.derivada_quarta_ordem(self.limite_sup)
        maior = self.saber_maior(d4_inf, d4_sup)
        parcela_1 = ((self.h ** 4) * (self.limite_sup - self.limite_inf)) / 180
        return abs(maior * parcela_1)

    def erro_generalizado(self):
        return self.erro_simples()

### Exemplo de uso com pontos

In [19]:
# Questão velocidade - Lista

pontos_x = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
pontos_y = [0, 50.4, 53.7, 67.5, 74.3, 84.6, 92.1, 98.3, 100, 105, 110]

simpson_13 = IntegralUmTercoSimpson(x=pontos_x, y=pontos_y)
print(f"Simpson 1/3 Margem 1 (x, y): {simpson_13.integral():.5f}")
print(f"Erro simples: {simpson_13.erro_simples():.5f}")
print(f"Erro generalizado: {simpson_13.erro_generalizado():.5f}")

print(f"Simpson 1/3 Margem M2 resultante (x, y): {(simpson_13.integral() + 2665.00000):.5f}")


Simpson 1/3 Margem 1 (x, y): 791.13333
Erro simples: 488.54093
Erro generalizado: 488.54093
Simpson 1/3 Margem M2 resultante (x, y): 3456.13333


In [20]:
# Questão 1 revisão - cos(x)**2 3 pontos e 5 pontos com simpson 1/3

# 3 pontos
def f(x):
    return math.cos(x)**2

simpson_38_2 = IntegralUmTercoSimpson(limite_inf=0, limite_sup=1, func_integral=f, h=1/2)
print(f"Simpson 1/3 (função): {simpson_38_2.integral():.5f}")
print(f"Erro simples: {simpson_38_2.erro_simples():.5f}")
print(f"Erro generalizado: {simpson_38_2.erro_generalizado():.5f}")

# 5 pontos
def f2(x):
    return math.cos(x)**2

simpson_38_2 = IntegralUmTercoSimpson(limite_inf=0, limite_sup=1, func_integral=f2, h=1/4)
print(f"Simpson 1/3 (função): {simpson_38_2.integral():.5f}")
print(f"Erro simples: {simpson_38_2.erro_simples():.5f}")
print(f"Erro generalizado: {simpson_38_2.erro_generalizado():.5f}")

Simpson 1/3 (função): 0.72876
Erro simples: 0.00000
Erro generalizado: 0.00000
Simpson 1/3 (função): 0.72741
Erro simples: 0.00009
Erro generalizado: 0.00009


### Exemplo de uso com função

In [21]:
# Questão 2: integral de 0 a 1 3*exp(-x) - Lista
def f(x):
    return 3 * math.exp(-x)

simpson_13_2 = IntegralUmTercoSimpson(limite_inf=0, limite_sup=1, func_integral=f, h=1/6)
print(f"Simpson 1/3 (x, y): {simpson_13_2.integral():.4f}")
print(f"Erro simples: {simpson_13_2.erro_simples():.4f}")
print(f"Erro generalizado: {simpson_13_2.erro_generalizado():.4f}")


Simpson 1/3 (x, y): 1.8964
Erro simples: 0.0000
Erro generalizado: 0.0000


In [22]:
x = sp.Symbol('x')
f_expr = sp.sqrt(x)

simpson = IntegralUmTercoSimpson(
    limite_inf=1,
    limite_sup=4,
    func_integral=f_expr,
    erro_desejado=0.01
)

print(f"Simpson 1/3 (x, y): {simpson.integral():.4f}")
print(f"Erro simples: {simpson.erro_simples():.4f}")
print(f"Erro generalizado: {simpson.erro_generalizado():.4f}")

valor de n (subintervalos) encontrado: 4
valor de h calculado: 0.75
Simpson 1/3 (x, y): 4.6662
Erro simples: 0.0003
Erro generalizado: 0.0003


### AV3

In [6]:
# Questão 2 AV3: integral de 0 a 1 para x * math.sqrt((x**2) + 1) utilizando n (numero de aplicações da regra) = 2, 10 e 50
def f(x):
    return x * math.sqrt((x**2) + 1)

limite_sup = 1
limite_inf = 0
n = 4
h = (limite_sup - limite_inf) / n

simpson = IntegralUmTercoSimpson(limite_inf=0, limite_sup=1, func_integral=f, h=h)
print(f"para n = {n}")
print(f"Simpson 1/3 (x, y): {simpson.integral():.4f}")
print(f"Erro simples: {simpson.erro_simples():.4f}")
print(f"Erro generalizado: {simpson.erro_generalizado():.4f}")

n = 6
h = (limite_sup - limite_inf) / n

simpson = IntegralUmTercoSimpson(limite_inf=0, limite_sup=1, func_integral=f, h=h)
print(f"para n = {n}")
print(f"Simpson 1/3 (x, y): {simpson.integral():.4f}")
print(f"Erro simples: {simpson.erro_simples():.4f}")
print(f"Erro generalizado: {simpson.erro_generalizado():.4f}")

n = 20
h = (limite_sup - limite_inf) / n

simpson = IntegralUmTercoSimpson(limite_inf=0, limite_sup=1, func_integral=f, h=h)
print(f"para n = {n}")
print(f"Simpson 1/3 (x, y): {simpson.integral():.4f}")
print(f"Erro simples: {simpson.erro_simples():.4f}")
print(f"Erro generalizado: {simpson.erro_generalizado():.4f}")

para n = 4
Simpson 1/3 (x, y): 0.6094
Erro simples: 0.0001
Erro generalizado: 0.0001
para n = 6
Simpson 1/3 (x, y): 0.6095
Erro simples: 0.0000
Erro generalizado: 0.0000
para n = 20
Simpson 1/3 (x, y): 0.6095
Erro simples: 0.0000
Erro generalizado: 0.0000


In [5]:
# Questão 3 AV3: área da figura, possui 2 áreas a ser calculadas que devem ser somadas

pontos_x = [0, 6, 12, 18, 24, 30, 36, 42, 48]
pontos_y = [0, 3, 6, 9, 10, 9, 8, 6, 0]

simpson_13 = IntegralUmTercoSimpson(x=pontos_x, y=pontos_y)
print(f"Simpson 1/3 área de cima (x, y): {simpson_13.integral():.4f}")
print(f"Erro simples: {simpson_13.erro_simples():.4f}")
print(f"Erro generalizado: {simpson_13.erro_generalizado():.4f}")

pontos_x = [0, 8, 16, 24, 32, 40, 48]
pontos_y = [0, 4, 5, 9, 8, 7, 0]

simpson_13_2 = IntegralUmTercoSimpson(x=pontos_x, y=pontos_y)
print(f"Simpson 1/3 área de baixo (x, y): {simpson_13_2.integral():.4f}")
print(f"Erro simples: {simpson_13_2.erro_simples():.4f}")
print(f"Erro generalizado: {simpson_13_2.erro_generalizado():.4f}")

print(f"Área Total = {(simpson_13.integral() + simpson_13_2.integral()):.4f}")

Simpson 1/3 área de cima (x, y): 312.0000
Erro simples: 27.6933
Erro generalizado: 27.6933
Simpson 1/3 área de baixo (x, y): 282.6667
Erro simples: 56.6667
Erro generalizado: 56.6667
Área Total = 594.6667
